# Gru GAN 1024

## Import

In [1]:
import argparse
import os
import numpy as np
import math
import pickle
import cv2 as cv
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
from torch.cuda.amp import autocast, GradScaler

import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision

## Hyperparameters

In [2]:
n_epochs = 100 # type=int, "number of epochs of training"
batch_size = 1 # type=int, "size of the batches"

lr = 0.002 # type=float "adam: learning rate"
b1 = 0.9 # type=float "adam: decay of first order momentum of gradient"
b2 = 0.999 # type=float "adam: decay of first order momentum of gradient"

num_gpu = 2 
cuda = torch.cuda.is_available()

latent_dim = 4 # type=int "dimensionality of the latent space"
img_size = 1024 # type=int "size of each image dimension"
channels = 1 # type=int "number of image channels"
sample_interval = 10000 # int "interval betwen image samples"

FLenTar = 100

dataset_dir = r"C:\Users/Leo's PC/Documents/SSTP Tests/SSTP/GruGan/test_frames"

## Datasets

In [3]:
class Dataset(Dataset):

    def __init__(self, file_dir, transform=None):

        self.dir = file_dir
        self.transform = transform
        self.diction = {}
        
        idx = 0
        for filename in os.listdir(self.dir):
            if filename.endswith('jpg'):
                self.diction[idx] = filename
                idx += 1
                        
    def __len__(self):
        return len(self.diction) - 1

    
    def __getitem__(self, idx):
        x, y = self.diction[idx], self.diction[idx + 1]
        directory_x, directory_y = self.dir + "\\" + str(x), self.dir + "\\" + str(y)
        x, y = cv.imread(directory_x), cv.imread(directory_y)
        if self.transform:
            x, y = self.transform(x), self.transform(y)
        return [x, y]

    
def HWC2CHW(x):
    return np.array(x).transpose(2, 0, 1)


dataset = Dataset(file_dir=dataset_dir, transform=HWC2CHW)

## Dataloaders

In [4]:
loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, drop_last=True)

## Model classes

In [63]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        #expected inout size (N, 3, 256, 128)
        
        #=======
        #Encoder
        #=======
        
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=[3,3], stride=1, padding=1)
        self.activation1_1 = nn.ReLU()
        self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=[3,3], stride=1, padding=1)
        self.activation1_2 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size = [2, 2], stride=2, padding=0)
        
        self.conv2_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=[3,3], stride=1, padding=1)
        self.activation2_1 = nn.ReLU()
        self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=[3,3], stride=1, padding=1)
        self.activation2_2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size = [2, 2], stride=2, padding=0)
        
        self.conv3_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=[3,3], stride=1, padding=1)
        self.activation3_1 = nn.ReLU()
        self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=[3,3], stride=2, padding=1)
        self.activation3_2 = nn.ReLU()
        self.conv3_3 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=[3,3], stride=1, padding=1)
        self.activation3_3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size = [2, 2], stride=2, padding=0)
        
        self.conv4_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation4_1 = nn.ReLU()
        self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=2, padding=1)
        self.activation4_2 = nn.ReLU()
        self.conv4_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation4_3 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size = [2, 2], stride=2, padding=0)
        
        self.conv5_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation5_1 = nn.ReLU()      
        self.conv5_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=2, padding=1)
        self.activation5_2 = nn.ReLU()
        self.conv5_3 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation5_3 = nn.ReLU()
        
        #size = (N, 512, 4, 2)
        
        #=======
        #Recurrent Module
        #=======
        
        self.pool_r = nn.AdaptiveMaxPool2d(output_size = (2, 1))
        
        #size of GRU input = (batch_size, seq_len, inp_size)
        self.gru = nn.GRU(input_size=1024, hidden_size=1024, num_layers=2, bias=True, batch_first=True, dropout=0.2)
        
        self.upsample_r = nn.Upsample(size=(8, 4))
        
        #=======
        #Decoder
        #=======
        
        #size = (N, 512, 8, 4)
        
        self.conv6_1 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation6_1 = nn.ReLU()
        self.conv6_2 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation6_2 = nn.ReLU()
        self.conv6_3 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation6_3 = nn.ReLU()
        self.upsample6 = nn.Upsample(scale_factor=2)
        
        self.conv7_1 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation7_1 = nn.ReLU()
        self.conv7_2 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation7_2 = nn.ReLU()
        self.conv7_3 = nn.ConvTranspose2d(in_channels=512, out_channels=512, kernel_size=[3,3], stride=1, padding=1)
        self.activation7_3 = nn.ReLU()
        self.upsample7 = nn.Upsample(scale_factor=2)
        
        self.conv8_1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=[3,3], stride=1, padding=1)
        self.activation8_1 = nn.ReLU()
        self.conv8_2 = nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=[3,3], stride=1, padding=1)
        self.activation8_2 = nn.ReLU()
        self.conv8_3 = nn.ConvTranspose2d(in_channels=256, out_channels=256, kernel_size=[3,3], stride=1, padding=1)
        self.activation8_3 = nn.ReLU()
        self.upsample8 = nn.Upsample(scale_factor=2)
        
        self.conv9_1 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=[3,3], stride=1, padding=1)
        self.activation9_1 = nn.ReLU()
        self.conv9_2 = nn.ConvTranspose2d(in_channels=128, out_channels=128, kernel_size=[3,3], stride=1, padding=1)
        self.activation9_2 = nn.ReLU()
        self.upsample9 = nn.Upsample(scale_factor=2)
        
        self.conv10_1 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=[3,3], stride=1, padding=1)
        self.activation10_1 = nn.ReLU()
        self.conv10_2 = nn.ConvTranspose2d(in_channels=64, out_channels=64, kernel_size=[3,3], stride=1, padding=1)
        self.activation10_2 = nn.ReLU()
        
        self.output = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=[1,1], stride=1, padding=0)
        self.activation_output = nn.Sigmoid()
        
        self.h = torch.Tensor(np.random.randn(2, 2, 1024)) #(num_layers, batchsize, inp.shape)
        

    def forward(self,x):
        
        #=======
        #Encoder
        #=======
        
        x = self.conv1_1(x)
        x = self.activation1_1(x)
        x = self.conv1_2(x)
        x = self.activation1_2(x)
        x = self.pool1(x)
        
        x = self.conv2_1(x)
        x = self.activation2_1(x)
        x = self.conv2_2(x)
        x = self.activation2_2(x)
        x = self.pool2(x)
        
        x = self.conv3_1(x)
        x = self.activation3_1(x)
        x = self.conv3_2(x)
        x = self.activation3_2(x)
        x = self.conv3_3(x)
        x = self.activation3_3(x)
        x = self.pool3(x)
        
        x = self.conv4_1(x)
        x = self.activation4_1(x)
        x = self.conv4_2(x)
        x = self.activation4_2(x)
        x = self.conv4_3(x)
        x = self.activation4_3(x)
        x = self.pool4(x)
        
        x = self.conv5_1(x)
        x = self.activation5_1(x)       
        x = self.conv5_2(x)
        x = self.activation5_2(x)
        x = self.conv5_3(x)
        x = self.activation5_3(x)
        
        #=======
        #Recurrent Module
        #=======
        
        
        x = self.pool_r(x)
        
        x = x.view(x.shape[0], -1)
        x = torch.unsqueeze(x, 1)
        
        x, self.h = self.gru(x, self.h)
        
        x = x[:,-1]
        x = torch.reshape(x, (x.shape[0], 512, 2, 1))
        
        x = self.upsample_r(x)
        
        #=======
        #Decoder
        #=======
        
        x = self.conv6_1(x)
        x = self.activation6_1(x)
        x = self.conv6_2(x)
        x = self.activation6_2(x)
        x = self.conv6_3(x)
        x = self.activation6_3(x)
        x = self.upsample6(x)
        
        x = self.conv7_1(x)
        x = self.activation7_1(x)
        x = self.conv7_2(x)
        x = self.activation7_2(x)
        x = self.conv7_3(x)
        x = self.activation7_3(x)
        x = self.upsample7(x)
        
        x = self.conv8_1(x)
        x = self.activation8_1(x)
        x = self.conv8_2(x)
        x = self.activation8_2(x)
        x = self.conv8_3(x)
        x = self.activation8_3(x)
        x = self.upsample8(x)
        
        x = self.conv9_1(x)
        x = self.activation9_1(x)
        x = self.upsample9(x)
        x = self.conv9_2(x)
        x = self.activation9_2(x)
        x = self.upsample9(x)
        
        x = self.conv10_1(x)
        x = self.activation10_1(x)
        x = self.conv10_2(x)
        x = self.activation10_2(x)
        
        x = self.output(x)
        x = self.activation_output(x)
        
        return x
    
    
    def name(self):
        return "Generator"
    
    
    def load_SalGan_weights(self, pretrained_weights='gen_modelWeights0090.npz'):
        
        self.pretrained_weights = np.load(open(pretrained_weights, 'rb'), allow_pickle=True)
        
        self.layers = [module for module in self.modules() if type(module) != nn.Sequential]
        self.conv_layers = [conv for conv in self.layers if type(conv) == nn.modules.conv.Conv2d or type(conv) == nn.modules.conv.ConvTranspose2d]

        array_idx = 0
        for layer in self.conv_layers:

            if type(layer) == nn.modules.conv.ConvTranspose2d:
                #the dim order of weight shape is messed up for ConvTranspose, so do transpose before loading

                self.reshaped_array = self.pretrained_weights['arr_' + str(array_idx)].transpose(1, 0, 2, 3)

                if layer.weight.shape == self.reshaped_array.shape:
                    layer.weight = nn.Parameter(torch.from_numpy(self.reshaped_array).float())
                else:
                    print("One layer was initialized with Xavier due to shape mismatch")
                    torch.nn.init.xavier_uniform_(layer.weight)           
                
                array_idx += 1  #add 1 to idx to fetch the next element from the file

                self.reshaped_array = self.pretrained_weights['arr_' + str(array_idx)]
                
                if layer.bias.shape == self.reshaped_array.shape:
                    layer.bias = nn.Parameter(torch.from_numpy(self.reshaped_array.copy()).float())
                else:
                    layer.bias.data.fill_(0.01)

            else: 
                layer.weight = nn.Parameter(torch.from_numpy(self.pretrained_weights['arr_' + str(array_idx)]).float())

                array_idx += 1  #add 1 to idx to fetch the next element from the file

                layer.bias = nn.Parameter(torch.from_numpy(self.pretrained_weights['arr_' + str(array_idx)]).float())

            array_idx += 1 #add 1 to idx to fetch the next element from the file
    
    
    
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.ResNet = torchvision.models.densenet121(pretrained=True)
        self.ResNet.fc = nn.Linear(in_features=1000, out_features=1000, bias=True)
        
        self.gru = nn.GRU(input_size=1000, hidden_size=512, num_layers=3, bias=True, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(512, 2, bias=True)
        
        self.softmax = nn.Softmax(dim=1)

    @autocast()
    def forward(self, x):
        
        try:
            self.h
        except:
            print(x.shape)
            self.h = torch.Tensor(np.random.randn(3, x.shape[0], 512)).cuda() #(num_layers, batchsize, inp.shape)
        '''
        x = self.ResNet(x)
        x = torch.unsqueeze(x, 1)
        x, self.h = self.gru(x, self.h)
        x = x[:,-1]
        x = self.fc(x)
        x = self.softmax(x)
        '''
        return Variable(Tensor([[0.3, 0.7]]))
    
    
    def name(self):
        return "Discriminator"
    
    
    def clear_mem(self):
        try:
            nn.init.xavier_uniform_(self.h)    
        except:
            print("Memory clear failed: hidden state does not exist.")
    
    
class StyleGanGenerator(nn.Module):
    def __init__(self):
        super(StyleGanGenerator, self).__init__()
        
        self.ResNet = torchvision.models.densenet121(pretrained=True)
        self.ResNet.fc = nn.Linear(in_features=1000, out_features=1000, bias=True)
            
        self.gru = nn.GRU(input_size=1000, hidden_size=512, num_layers=3, bias=True, batch_first=True, dropout=0.2)
        
        with open('ffhq256.pkl', 'rb') as f:
            self.StyleGan = pickle.load(f)['G_ema'].cuda() 
        
    #@autocast()    
    def forward(self, x):
        try:
            self.h
        except:
            self.h = torch.Tensor(np.random.randn(3, x.shape[0], 512)).cuda() #(num_layers, batchsize, inp.shape)

        ## ENCODER
        x = self.ResNet(x)
        
        ## GRU
        x = torch.unsqueeze(x, 1)
        x, self.h = self.gru(x, self.h)
        x = torch.squeeze(x, 1)
        
        ## DECODER
        # x = self.StyleGan(x, None)
        
        return x
        
    
    def name(self):
        return "StyleGanGenerator"
    
    
    def clear_mem(self):
        try:
            nn.init.xavier_uniform_(self.h)    
        except:
            print("Memory clear failed: hidden state does not exist.")

In [64]:
class LengthScheduler():
    def __init__(self, target_seq_len=100):
        self.target_len_final = target_seq_len
        self.current_seq_len_target = 2
        self.total_call_count = 0
        self.running_count = 0
        self.trained_count = 0
        self.loss_hist = np.array([])
    
    
    def approve(self, loss, current_seq_len):
        def return_true(update=True):
            self.total_call_count += 1
            self.running_count = 0
            self.trained_count += 1
            
            self.loss_hist = np.append(self.loss_hist, loss)
            
            if update: adjust_target_len()
            
            return True
        
        def return_false():
            self.total_call_count += 1
            self.running_count += 1
            
            return False
        
        def simple_len_counter():
            return True if self.running_count >= self.current_seq_len_target else False
        
        def adjust_target_len():
            self.dy = np.diff(self.loss_hist)
            
            try:
                self.running_dy = self.running_dy * 0.9 + self.dy[-1] * (1 - 0.9)
            except:
                self.running_dy = self.running_dy
                
            if self.running_dy > -0.00005:
                self.current_seq_len_target += 1
                print('training length changed to', self.current_seq_len_target)
        
        
        if self.total_call_count < 200:
            # for the first 200 inputs, always return True
            return return_true(update=False)
        else:
            return return_true() if self.running_count >= self.current_seq_len_target else return_false()
        
        
    def reset(self):
        self.total_call_count = 0
        self.running_count = 0
        self.trained_count = 0

## Loss, Optimizer, Training setup

In [65]:
# Loss function
adversarial_loss = torch.nn.BCEWithLogitsLoss()


# Initialize generator and discriminator
G = StyleGanGenerator()
D = Discriminator()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


G.cuda().to(device)
D.cuda().to(device)
adversarial_loss.cuda()


#G = torch.nn.DataParallel(G)
#D = torch.nn.DataParallel(D)


optimizer_G = torch.optim.Adam(G.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(D.parameters(), lr=lr, betas=(b1, b2))

scaler = GradScaler()

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
scheduler = LengthScheduler()

In [24]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

G.apply(init_weights)
D.apply(init_weights)

AttributeError: 'NoneType' object has no attribute 'data'

## Training

The Generator takes an input frame x, outputs a fake frame y_hat. If we calculate the loss and back_prop now, we are training a Generator that can predict the next frame. If a sequence of mutiple fake frames is desired during inference, we then feed the Generator y_hat as the input and get the next frame. However, here's a problem: during training, the model only learns how to produce a fake frame based on a real frame. If we want to countinuously feed the model hundreds of  fake frames, the quality of the output will deteriorate quickly.

Therefore, we train the model in a way that it's loss is judged based on the whole sequence, not just the next frame. In a sense, this trains the model in a setup that is closer to its use-case senerio. However, keep in mind that the video is still generated by passing in fake(t-1) to get fake(t), in a loop.

We propose a length scheduler that automatically determines the length of the output sequence based on which the generator will be judged. The generator will start with learning how to predict the next frame. When it gets good at it, it will learn to generate 2 frames, untill the target value is hit.

In [72]:
y_hat_list = []
y_list = []
rating = 0
loss = 1
torch.autograd.set_detect_anomaly(True)
    
for i, x in enumerate(loader):
    
    x, y = Variable(x[0].type(Tensor)).cuda().to(device), Variable(x[1].type(Tensor)).cuda().to(device)
    
    #with autocast():
    y_hat = G(x)
        
    y_hat_list.append(y_hat.cpu().detach().numpy())
    
    ## Evaluation and backprop
    
    if scheduler.approve(loss, len(y_hat_list)): #returns True if it approves of evaluation
        
        ## ========================
        ## Train the Generator
        ## ========================
        
        D.clear_mem()
        optimizer_G.zero_grad()
        
        with autocast():
            for fake in y_hat_list:    
                fake = Variable(Tensor(fake)).cuda().to(device)
                rating = D(fake)
                print('hello')
            
        ad_loss = adversarial_loss(rating, Variable(Tensor([1.0, 0.0])).unsqueeze(0))
        gt_loss = adversarial_loss(y_hat, Tensor(np.random.randn(1, 512)))

        g_loss = 0.7 * ad_loss + 0.3 * gt_loss
        
        g_loss.backward(retain_graph=True)
        optimizer_G.step()
        
        y_hat_list = []
        y_list = []
        x_list = []
        
        G.clear_mem()
        
        ## ========================
        ## Train the Discriminator
        ## ========================
        
        D.clear_mem()
        optimizer_D.zero_grad()
        
        with autocast():
            rating = D(x)
            for y_hat in y_hat_list:           
                rating = D(y_hat)
            fake_loss = adversarial_loss(rating, Variable(Tensor([0.0, 1.0])).unsqueeze(0))
        
            D.clear_mem()
            
            rating = D(x)
            for y in y_list:
                rating = D(y)
            real_loss = adversarial_loss(rating, Variable(Tensor([1.0, 0.0])).unsqueeze(0))

            d_loss = 0.5 * real_loss + 0.5 * fake_loss
        
        #scaler.scale(d_loss).backward()
        #scaler.step(optimizer_D) 
        #scaler.update()

hello


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [1536, 1000]] is at version 2; expected version 1 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [ ]:
'''

checkpoint_file = open(r"C:/Users/Leo's PC/Documents/SSTP Tests/Chinese Characters/LightGAN out/G.tar", 'wb')
torch.save({'model': G.state_dict()}, checkpoint_file)
checkpoint_file.close()

checkpoint = torch.load(open("C:/Users/Leo's PC/Documents/SSTP Tests/Chinese Characters/LightGAN out/G.tar", 'rb'))
G.load_state_dict(checkpoint['model'])

'''